<a href="https://colab.research.google.com/github/nutte2/BERT/blob/master/Vatten1TRFV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Testprogram för att köra Vattenanmälningar från Trafikverket/Sweco till Länsstyrelsen.
Kalle Prorok 2021-04-26




In [2]:
#Utilities for NLP. Kalle Prorok 2021
#Använder Kungliga bibliotekets BERT
#Dolda koden normalt placerad i NLPUtils




In [1]:
#@title
#to be placed in NLPUTIL
# urllib

def frånWebsida(url,filnamn):
  geturllibrequest(url+filnamn)
  return fil



In [3]:
#Testprogram
#import nlputil # Kalles AI-rutiner, inte klara än :)

def vattenanmälan():
    # Hämta lämpliga dokument
    url= 'https://www.trafikverket.se/'# sajt
    # Man kan nog (annars) söka på ordet Miljökonsekvensbeskrivning för att hitta rätt dokument
    mkb=frånWebsida(url,'contentassets/e4edb40590bc47fcae6541b8fb0a17be/miljokonsekvensbeskrivning5.pdf')
    PMAvvattning=frånWebsida()
    PMMarkmiljö=frånWebsida()
    PMNaturvärdesinventering=frånWebsida()
    planbeskrivning=frånWebsida(url,'contentassets/e4edb40590bc47fcae6541b8fb0a17be/180704_planbeskrivning.pdf')
    översiktskarta=frånWebsida(url,'contentassets/e4edb40590bc47fcae6541b8fb0a17be/180704_oversiktskarta.pdf')

    # Se vilka bäckar som finns nämnda och kan vara aktuella, gör en ansökan för varje
    # ner=named entity recognition, letar efter platser med ordet bäck i sig, man kan ha flera varianter
    bäckar=ner(mkb,'POS','*bäck*||*ån*||*älv')
    for bäck in bäckar:
        ansökan=formulär('Länsstyrelse vattenanmälan v1',bäck) # Skapa grundformulär

        #Fyll formuläret med massa fakta via add (addera)
        ansökan.add('Länsstyrelse',ner(mkb,'ORG','*länsstyrelse*'))
        ansökan.add('Kommun',ner(mkb,'POS','*kommun'))
        # hitta mer eller mindre googlar fram resultat
        ansökan.add('Organisationsnummer',hitta(['organisationsnummer',ansökan.data('Länsstyrelse')))
        ansökan.add('Typ av verksamhet','Byte av trumma')

        #hittafakta använder frågor&svar för att hitta info
        ansökan.add('Medelvattenföring MQ',hittafakta(PMAvvattning,['Medelvattenföring' bäck])
        ansökan.add('Trummans diameter',hittafakta(PMAvvattning,['Trummans diameter' bäck])
        ansökan.add('Typ av vatten', 'Ytvatten')

        #sammanfatta använder BERT för att hitta viktiga meningar i ett stycke där
        # sökorden eller ord med liknande betydelse finns
        ansökan.add('Beskrivning', sammanfatta(mkb,[bäck 'beskrivning'])
        ansökan.add('Beskriv omgivning', sammanfatta(mkb, [bäck 'beskrivning' 'omgivning'])

        ansökan.add('Föroreningar',hittafakta(PMMarkmiljö,'Föroreningar'))
        ansökan.add('Vilket vattendrag',bäck)
        ansökan.add('Koordinater',bäck) # behöver ofta matas in manuellt

        #figur letar fram en bild i ett dokument med egenskaper som känns igen av ett
        # AI-system, google translate (typ-ordet karta till map) + deep learning (image_net) här
        ansökan.add('Översiktskarta',figur(mkb,['karta','legend'],sida=1:5))
        ansökan.add('Detaljkarta',figur(mkb,['karta','legend'],sida=6:))
        ansökan.add('Ritning',figur(mkb,['ritning','legend']))

        #redigera presenterar det man kommit fram till för ändringar och kompletteringar
        #kanske är via Anteckningar eller möjligen Word.
        #Bildfiler ligger separat för inspektion/val/ändring/uppladdning
        if ansökan.redigera()): # Om redigeringen är klar/ok så skickas den in
            if ansökan.fylli(): # Fyller i Länsstyrelsens formulär
                if ansökan.ok(): # om det verkar rätt ifyllt
                    if ansökan.betala('betaluppgifter'): # så betala
                        ansökan.klar()




SyntaxError: ignored